# Split data

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
TRAIN_SIZE = 0.9
TEST_SIZE = 0.0
VAL_SIZE = 0.1
RANDOM_SEED = 911

In [3]:
train_df = pd.read_csv("./data/train.csv")
train_df.head()

,ID,set,question,answer,score
0,0,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรจะมีค่าต่ำ เพราะว่าเราต้องการทร...,0.0
1,1,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เพราะเป็นการเก็บข้อมูลจากหลาย users ...,5.0
2,2,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรเป็น #checkout events/ #cookies...,5.0
3,3,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,"เห็นด้วย ให้ X~Binomial(N,p) โดย p เป็นอัตราส่...",2.0
4,4,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เห็นด้วย เนื่องจากการทดสอบ A/B Testing เป็นวิธ...,4.5


In [4]:
train_df, val_df = train_test_split(train_df, test_size=VAL_SIZE, random_state=RANDOM_SEED)
if TEST_SIZE > 0:
    train_df, test_df = train_test_split(train_df, test_size=TEST_SIZE, random_state=RANDOM_SEED)

print(f"Train size: {len(train_df)}")
print(f"Validation size: {len(val_df)}")
print(f"Test size: {len(test_df) if TEST_SIZE > 0 else 0}")

Train size: 325
Validation size: 37
Test size: 0


# Create model

In [ ]:
GROQ_API = ""

In [6]:
train_df

,ID,set,question,answer,score
75,75,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,Hamtaro should consider using the binomial dis...,5.0
232,232,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ตามชั่วโมงหรือวัน เพื่อจับความแปรป...,0.0
274,274,Q3,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,\nการเลือกใช้ binomial distribution ใน A/B tes...,3.0
263,263,Q4,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,สัดส่วน 50/50 ทำให้เราสามารถเก็บข้อมูลและเห็นค...,5.0
250,250,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,ตัวเศษ - จำนวนครั้งผู้ใช้ที่คลิกโฆษณา\nตัวส่วน...,4.5
...,...,...,...,...,...
72,72,Q4,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,การเลือกสัดส่วนของผู้ใช้ที่เห็นโฆษณาตำแหน่งเก่...,3.0
95,95,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,A/B testing คือเครื่องมือที่เหมาะสมในการทดสอบก...,4.5
283,283,Q1,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,เพราะการนำ A/B testing มาใช้โดยให้ A test เป็น...,1.0
60,60,Q2,Hamtube เป็นแพลตฟอร์มดูวีดีโอออนไลน์ ที่อนุญาต...,granularity ควรเป็น ads clicked/ unique user เ...,5.0


In [7]:
# Create system promt
base_system_promt = """
Act as a professor in Computer Engineering Mathematics class and grading the student answer, You will grading the student answer. 
The student answer is decribing the understanding in the question. The score is between 0 to 5 e.g. 0, 1.5, 2, 3.5, etc.
Your Answer must be only number between 0 to 5 and has no other characters.

The question is: {question}
"""


def get_few_shot_promt(set_id: int):
    few_short = []
    explored_score = set()
    for i, row in train_df[train_df["set"] == f"Q{set_id}"].iterrows():
        if row["score"] // 1 not in explored_score:
            explored_score.add(row["score"] // 1)
            few_short.append(
                {
                    "role": "user",
                    "content": f"Answer: {row['answer']}",
                }
            )
            few_short.append(
                {
                    "role": "assistant",
                    "content": f"{row['score']}",
                }
            )
    return few_short


get_few_shot_promt(1)

[{'role': 'user',
  'content': 'Answer: A/B testing ช่วยให้แฮมทาโร่สามารถวัดผลทางการตลาดได้อย่างมีประสิทธิภาพเท่าที่จะเป็นไปได้ โดยในที่นี้คือการวัดผลจากการย้ายตำแหน่งโฆษณาที่เป็นจุดสนใจ. นั่นจะช่วยให้เขาทราบว่าการเปลี่ยนแปลงนี้มีผลต่อการคลิก, การแชร์, หรือยอดขายหรือไม่ โดย A/B testing สามารถแบ่งกลุ่มเป้าหมายได้อย่างเท่าเทียม และ เปรียบเทียบในเชิงสถิติได้'},
 {'role': 'assistant', 'content': '4.5'},
 {'role': 'user',
  'content': 'Answer: Hamtaro ควรใช้ A/B testing เพราะช่วยให้ Hamtaro รวบรวมข้อมูลการโต้ตอบของผู้ใช้กับตำแหน่งโฆษณาต่างๆ ได้ ช่วยให้เขาตัดสินใจได้อย่างมีข้อมูลโดยพิจารณาจากพฤติกรรมของผู้ใช้จริง นอกจากนี้ยังให้หลักฐานที่เป็นรูปธรรมเกี่ยวกับผลกระทบของการย้ายตำแหน่งของโฆษณาที่มีต่อinteractioinของผู้ใช้ด้วย'},
 {'role': 'assistant', 'content': '1.5'},
 {'role': 'user',
  'content': 'Answer: ควร เนื่องจากการทำ A/B Testing จะทำให้ได้\n1. Data-Driven Decisions ไม่ใช่การ "เดา"\n2. Risk Mitigation, หากเปลี่ยนตำแหน่งโฆษณาทั้งหมดเลยอาจทำให้ผลประกอบการ drop ได้ การทำ A/B จะทำให้ได้ลอง

In [8]:
from groq import Groq
from tqdm import tqdm
import os
import re

client = Groq(api_key=GROQ_API)

In [9]:
answer_pattern = re.compile(r"\d+\.?\d*")

In [10]:
test_df = pd.read_csv("./data/test.csv")
if os.path.exists("pure_llm.csv"):
    submission_df = pd.read_csv("pure_llm.csv")
else:
    submission_df = pd.DataFrame(columns=["ID", "score"], dtype=float)
    submission_df["ID"] = test_df["ID"]

submission_df["ID"] = submission_df["ID"].astype(int)
submission_df.set_index("ID", inplace=True)

In [11]:
looper = tqdm(test_df.iterrows(), total=len(test_df))
for row in looper:
    row = row[1]
    if not pd.isna(submission_df.loc[int(row["ID"]), "score"]):
        continue
    test_sample = f"<Answer>\n{row['answer']}\n</Answer>"
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": base_system_promt.format(question=row["question"])},
            *get_few_shot_promt(row["ID"]),
            {"role": "user", "content": test_sample},
        ],
        temperature=1e-5,
        max_completion_tokens=1024,
        top_p=1e-5,
        stream=False,
        stop=None,
    )
    llm_answer = completion.choices[0].message.content
    try:
        score = float(answer_pattern.findall(llm_answer)[0])
        if score > 5:
            score = 5
        elif score < 0:
            score = 0
        submission_df.loc[row["ID"], "score"] = score
        looper.set_description(f"ID: {row['ID']} - Score: {score}")
    except:
        print(f"Error: {llm_answer}")
        print(f"{completion}")

    submission_df.to_csv("pure_llm.csv", index=True)

ID: 451 - Score: 4.5: 100%|██████████| 90/90 [00:45<00:00,  1.98it/s]
